In [1]:
# Importing important libraries
from tkinter import *
from tkinter import font
import cv2
import mediapipe
from PIL import Image, ImageTk
import numpy as np
import os
from tensorflow.keras.models import load_model
from pynput.keyboard import Key, Controller
import random
from playsound import playsound
import threading
import imageio

In [2]:
# Defining a drawing module to track the hand
drawingModule = mediapipe.solutions.drawing_utils
handsModule = mediapipe.solutions.hands
keyboard = Controller()
capture = cv2.VideoCapture(0)

frameWidth = 640
frameHeight = 480

# Defining a TKinter module
master = Tk()
master.wm_title("Learn Alphabets")
master.config(background="#FFFFFF")
fontStyle = font.Font(family="Lucida Grande", size=20)
fontStyleHeader = font.Font(family="Lucida Grande", size=10)

# Loading the saved ML model 
savedModel=load_model('./model/model.h5')

In [3]:
# Defining global variables
key = None
flag = False
s_flag = False
continue_flag = True
next_flag = False
thread_running = False

answer = np.zeros((int(frameHeight), int(frameWidth), 3), dtype=np.uint8)
pts = []

pred = StringVar()
pred.set('Prediction: ')
toDraw = StringVar()
toDraw.set('To Draw: ')
evaluation_answer = StringVar()
game_over = StringVar()

# Defining Keypress event to navigate between the app
def keypress(event):
    global key, flag, s_flag, next_flag, pts, answer, continue_flag
    key = event.char
    if(key == 'n' and next_flag):
        button3.config(state="normal")
        reset_frame()
    if(key == 's'):
        flag = not(flag)
        if (s_flag):
            prediction()
        s_flag = not(s_flag)
    if(key == 'r'):
        if(not(thread_running)):
            thread = threading.Thread(target=stream, args=(my_label,))
            thread.daemon = 1
            thread.start()
    if(key == 'z'):
        answer = np.zeros((int(frameHeight), int(frameWidth), 3), dtype=np.uint8)
        pts = []
        flag = False
        s_flag = False
        continue_flag = True
        next_flag = False
        if(not(thread_running)):
            thread = threading.Thread(target=stream, args=(my_label,))
            thread.daemon = 1
            thread.start()
    
master.bind("<KeyPress>", keypress)

'1971144055552keypress'

In [ ]:
# Defining additional global variables  
random_char = None
change_flag = False
video = None
location = None

# Defining a function to get the location of the video of the random alphabet
def getvideo(alphabet):
    global video, location
    location = "./alphabets/"+alphabet
    video = imageio.get_reader(location+'.mp4')

# Defining a function to play the video of the random alphabet
def stream(label):
    global location,video, thread_running
    thread_running = True
    button1.config(state="disabled")
    playsound(location+'.mp3')
    for image in video.iter_data():
        frame_image = ImageTk.PhotoImage(Image.fromarray(image))
        label.config(image=frame_image)
        label.image = frame_image
    enable_thread()

# Defining a function to enable thread
def enable_thread():
    global thread_running
    thread_running = False
    button1.config(state="normal")

# Defining a function to generate random characters
def generate_random_char():
    global random_char
    a = random.randrange(len(supported_chars))
    random_char = str(supported_chars[a])
    getvideo(random_char)
    b = "To Draw: "+ (random_char)
    toDraw.set(b)

# Special condition for the characters which look alike
def condition(drawn):
    global random_char
    if(random_char == drawn):
        return True
    elif(random_char == 'S' and drawn == '5'):
        return True
    elif(random_char == '5' and drawn == 'S'):
        return True
    elif(random_char == '0' and (drawn == 'O' or drawn == 'D')):
        return True
    elif(random_char == 'O' and (drawn == '0' or drawn == 'D')):
        return True
    elif(random_char == 'D' and (drawn == '0' or drawn == 'O')):
        return True
    elif(random_char == 'Z' and drawn == '2'):
        return True
    elif(random_char == '2' and drawn == 'Z'):
        return True
    elif(random_char == '6' and drawn == 'G'):
        return True
    elif(random_char == 'G' and drawn == '6'):
        return True
    return False

# To check whether or not the air gesture is correct
def evaluate(drawn):
    global random_char, change_flag
    if(condition(drawn)):
        label4.config(bg="green")
        evaluation_answer.set('Correct!!!')
        supported_chars.remove(str(random_char))
        button3.config(state="disabled", text = 'Press Next')
        button1.config(text='Next')
        playsound('./audio/correct.mp3')
        if(len(supported_chars)==0):
            label5.config(bg="green")
            playsound('./audio/congrats.mp3')
            button1.config(state = "disabled", text = "Congrats!")
            button2.config(state = "disabled", text = "Congrats! You have Finished it! Relaunch to Try Again")
            button3.config(state = "disabled", text = "Congrats!")
            game_over.set('Congrats, You Finished it!')
        else:
            change_flag = True
    else:
        label4.config(bg="red")
        evaluation_answer.set('InCorrect!!! Try Again')
        button3.config(state="disabled", text = 'Press Try Again')
        button1.config(text = 'Try Again')
        playsound('./audio/wrong.mp3')
        
# Predict the air gesture drawn
def decode(num):
    return num if num <= 9 else chr(int(num) + 55)

def predict(img):
    global thread_running
    img = np.array(img)
    img = img.reshape((1, 28, 28, 1))
    img = img / 255.0
    Y = savedModel.predict([img])[0]
    a = str(decode(np.argmax(Y)))
    ans = "Prediction: "+ a
    pred.set(ans)
    evaluate(a)
    button1.config(state="normal")
    
def prediction():
    global answer,continue_flag, next_flag
    blackboard_gray = cv2.cvtColor(answer, cv2.COLOR_BGR2GRAY)
            
    blur1 = cv2.medianBlur(blackboard_gray, 15)
    blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)

    ret, thresh = cv2.threshold(
        blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    blackboard_cnts, blackboard_hier = cv2.findContours(
        thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    if len(blackboard_cnts) > 0:
        cnt = sorted(blackboard_cnts,
                     key=cv2.contourArea, reverse=True)[0]
        if cv2.contourArea(cnt) > 1000:
            x, y, w, h = cv2.boundingRect(cnt)
            alphabet = blackboard_gray[y -
                                       10:y + h + 10, x-10:x + w + 10]
            newImage = cv2.resize(alphabet, (28, 28))
            predict(newImage)
            continue_flag = not(continue_flag)
            next_flag = not(next_flag)

# To access the camera and track the index finger
def show_frame():
    if(not(continue_flag)):
            return

    with handsModule.Hands(static_image_mode=False, min_detection_confidence=0.6, min_tracking_confidence=0.6, max_num_hands=1) as hands:
        global pts,answer,flag
        _, frame = capture.read()
        frame = cv2.resize(frame,(640,480))
        frame = cv2.flip(frame, 1)
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.multi_hand_landmarks != None:
            normalizedLandmark = results.multi_hand_landmarks[0].landmark[handsModule.HandLandmark.INDEX_FINGER_TIP]
            pixelCoordinatesLandmark = drawingModule._normalized_to_pixel_coordinates(normalizedLandmark.x,
                                                                                          normalizedLandmark.y,
                                                                                          frameWidth,
                                                                                          frameHeight)

            cv2.circle(frame, pixelCoordinatesLandmark, 15, (255, 0, 0), cv2.FILLED)
            
            if(not(flag)):
                answer = np.zeros((int(frameHeight), int(frameWidth), 3), dtype=np.uint8)
                cv2.circle(answer, pixelCoordinatesLandmark, 15, (255, 255, 255), cv2.FILLED)
                
                    
            if(flag):
                answer = np.zeros((int(frameHeight), int(frameWidth), 3), dtype=np.uint8)

                pts.append(pixelCoordinatesLandmark)

        for i in range(1, len(pts)):
            cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), 15)
            cv2.line(answer, pts[i - 1], pts[i], (255, 255, 255), 15)
        
        mirrored_board = answer.copy()
        
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        display1.imgtk = imgtk #Shows frame for display 1
        display1.configure(image=imgtk)
        
        bcv2image = cv2.cvtColor(mirrored_board, cv2.COLOR_BGR2RGBA)
        bimg = Image.fromarray(bcv2image)
        bimgtk = ImageTk.PhotoImage(image=bimg)
        display2.imgtk = bimgtk #Shows frame for display 2
        display2.configure(image=bimgtk)
                      
        
    master.after(20, show_frame)

# To reset the frame
def reset_frame():
    global pts,answer,flag,s_flag,continue_flag, next_flag, change_flag
    flag = False
    s_flag = False
    continue_flag = True
    next_flag = False
    
    
    
    answer = np.zeros((int(frameHeight), int(frameWidth), 3), dtype=np.uint8)
    pts = []
    
    
    if (change_flag):
        generate_random_char()
        keyboard.press('r')
        button1.config(text='Next')
        button3.config(text='Reset')
        change_flag = False
    
    else:
        keyboard.press('r')
        button1.config(text="Try Again")
        button3.config(text='Reset')
    
    pred.set('Prediction: ')
    
    label4.config(bg="white")
    evaluation_answer.set('')
    
    button1.config(state="disabled")
    show_frame()

# additional functions to emulate button press
def press_n():
    keyboard.press("n")

def press_r():
    keyboard.press("r")

def press_z():
    keyboard.press("z")

# Defining a list of supported characters
supported_chars = list(map(chr, range(ord('A'), ord('Z')+1)))
supported_chars.extend(range(0,10))
supported_chars = [str(i) for i in supported_chars]

# Frame 1 of the TKinter - a button to play audio/video again
frame1 = Frame(master, width=2*frameWidth, height=frameHeight, bg="white")
frame1.grid(sticky="nsew", columnspan=2)
frame1.grid_columnconfigure(0, weight=1)
button2 = Button(frame1, text = 'Press "S" to Start/Stop, Click Here to Listen Again', command = press_r, font = fontStyleHeader)
button2.pack()

# Frame 2 of the TKinter - to play the audio/video of random character
frame2 = Frame(master, width=frameWidth, height=frameHeight/2)
frame2.grid(row=1,column=0)
my_label = Label(frame2)
my_label.pack()
thread = threading.Thread(target=stream, args=(my_label,))

# Frame 3 - Shows the random chracter and the prediction
frame3 = Frame(master, width=frameWidth, height=frameHeight/2)
frame3.grid(row=1,column=1)
label2 = Label(frame3, textvariable = toDraw, font = fontStyle, bg="white")
label2.pack(fill="x",expand = True)
label3 = Label(frame3, textvariable = pred, font = fontStyle, bg="white")
label3.pack(fill="x",expand = True)
label4 = Label(frame3, textvariable = evaluation_answer, font = fontStyle, bg="white")
label4.pack(fill="x",expand = True)
label5 = Label(frame3, textvariable = game_over, font = fontStyle, bg="white")
label5.pack(fill="x",expand = True)

# Frame 4 - Shows the user camera and tracks the finger
frame4 = Frame(master, width=frameWidth, height=frameHeight)
frame4.grid(row=2,column=0)
display1 = Label(frame4)
display1.grid(sticky="nsew")

# Frame 5 - Shows a blackboard - which shows only the finger movements
frame5 = Frame(master, width=frameWidth, height=frameHeight)
frame5.grid(row=2, column=1)
display2 = Label(frame5)
display2.grid(sticky="nsew")

# Frame 6 - Shows Next/ Try Again button
frame6 = Frame(master, width=frameWidth, height=frameHeight)
frame6.grid(row=3,column=0)
button1 = Button(frame6, text = 'Next', command = press_n, state = "disabled")
button1.pack()

# Frame 7 - Shows Reset button - to reset the user inputs
frame7 = Frame(master, width=frameWidth, height=frameHeight)
frame7.grid(row=3,column=1)
button3 = Button(frame7, text = 'Reset', command = press_z)
button3.pack()

# Runs the main loop
generate_random_char()
thread.daemon = 1
thread.start()
show_frame()
master.mainloop()
capture.release()